# Tahap cleaning fundamental ini
1. load semua file
2. rename symbol
3. ubah wide → long
4. merge semua metrik
5. hitung ROA, ROE, margin, dll
6. merge stats
7. merge valuation
8. saya ambil tahun 2022
9. simpan

In [6]:
import pandas as pd
import numpy as np

# Load semua file (asumsi file ada di folder kerja yang sama dengan notebook)
df_assets   = pd.read_csv("data/data_assets.csv")
df_cash     = pd.read_csv("data/data_cash.csv")
df_equity   = pd.read_csv("data/data_equity.csv")
df_profit   = pd.read_csv("data/data_profit.csv")
df_revenue  = pd.read_csv("data/data_revenue.csv")
df_stats    = pd.read_csv("data/data_stats.csv")
df_valuation = pd.read_csv("data/data_valuation.csv")

# Cek salah satu untuk memastikan
df_assets.head()


,Unnamed: 0,2022,2021,2020,2019
0,MSFT,3.648400e+11,3.337790e+11,3.013110e+11,2.865560e+11
1,AMZN,4.205490e+11,3.211950e+11,2.252480e+11,1.626480e+11
2,TSLA,6.213100e+10,5.214800e+10,3.430900e+10,2.974000e+10
3,GOOGL,3.592680e+11,3.196160e+11,2.759090e+11,2.327920e+11
4,GOOG,3.592680e+11,3.196160e+11,2.759090e+11,2.327920e+11


# ganti nama kolom unnamed, dari wide ke long

In [9]:
def reshape_year_table(df, metric_name):
    df = df.copy()
    # ganti nama kolom ticker
    df = df.rename(columns={"Unnamed: 0": "symbol"})
    
    # ubah wide -> long
    df_long = df.melt(
        id_vars=["symbol"],
        var_name="year",
        value_name=metric_name
    )
    
    # tahun jadi int
    df_long["year"] = df_long["year"].astype(int)
    # nilai jadi numerik
    df_long[metric_name] = pd.to_numeric(df_long[metric_name], errors="coerce")
    
    return df_long


In [10]:
assets_long   = reshape_year_table(df_assets,  "assets")
cash_long     = reshape_year_table(df_cash,    "cash")
equity_long   = reshape_year_table(df_equity,  "equity")
profit_long   = reshape_year_table(df_profit,  "profit")
revenue_long  = reshape_year_table(df_revenue, "revenue")

assets_long.head()


,symbol,year,assets
0,MSFT,2022,3.648400e+11
1,AMZN,2022,4.205490e+11
2,TSLA,2022,6.213100e+10
3,GOOGL,2022,3.592680e+11
4,GOOG,2022,3.592680e+11


# di merge untuk jadi 1 tabel

In [11]:
fundamental = assets_long.merge(cash_long,   on=["symbol", "year"], how="left")
fundamental = fundamental.merge(equity_long, on=["symbol", "year"], how="left")
fundamental = fundamental.merge(profit_long, on=["symbol", "year"], how="left")
fundamental = fundamental.merge(revenue_long,on=["symbol", "year"], how="left")

fundamental.head()


,symbol,year,assets,cash,equity,profit,revenue
0,MSFT,2022,3.648400e+11,1.393100e+10,1.665420e+11,135620000000,198270000000
1,AMZN,2022,4.205490e+11,3.622000e+10,1.382450e+11,197478000000,469822000000
2,TSLA,2022,6.213100e+10,1.757600e+10,3.018900e+10,13606000000,53823000000
3,GOOGL,2022,3.592680e+11,2.094500e+10,2.516350e+11,146698000000,257637000000
4,GOOG,2022,3.592680e+11,2.094500e+10,2.516350e+11,146698000000,257637000000


# hitung rasio keuangannya kaya ROA ROE MARGIN dll nya

In [14]:
fundamental["roa"] = fundamental["profit"] / fundamental["assets"]
fundamental["roe"] = fundamental["profit"] / fundamental["equity"]
fundamental["profit_margin"] = fundamental["profit"] / fundamental["revenue"]
fundamental["cash_ratio"] = fundamental["cash"] / fundamental["assets"]
fundamental["leverage"] = fundamental["assets"] / fundamental["equity"]

fundamental.head()



,symbol,year,assets,cash,equity,profit,revenue,roa,roe,profit_margin,cash_ratio,leverage
0,MSFT,2022,3.648400e+11,1.393100e+10,1.665420e+11,135620000000,198270000000,0.371725,0.814329,0.684017,0.038184,2.190679
1,AMZN,2022,4.205490e+11,3.622000e+10,1.382450e+11,197478000000,469822000000,0.469572,1.428464,0.420325,0.086126,3.042056
2,TSLA,2022,6.213100e+10,1.757600e+10,3.018900e+10,13606000000,53823000000,0.218989,0.450694,0.252792,0.282886,2.058068
3,GOOGL,2022,3.592680e+11,2.094500e+10,2.516350e+11,146698000000,257637000000,0.408325,0.582979,0.569398,0.058299,1.427735
4,GOOG,2022,3.592680e+11,2.094500e+10,2.516350e+11,146698000000,257637000000,0.408325,0.582979,0.569398,0.058299,1.427735


In [15]:
fundamental.replace([np.inf, -np.inf], np.nan, inplace=True) # kalau ada nan kita harus kita rapihkan karena pembagian yaitu 0

# bersihkan data stats sama valuation

In [16]:
# stats
df_stats = df_stats.rename(columns={"Unnamed: 0": "symbol"})
df_stats.columns = [c.strip().lower().replace(" ", "_") for c in df_stats.columns]
df_stats.head()

# valuation
df_valuation = df_valuation.rename(columns={"Unnamed: 0": "symbol"})
df_valuation.columns = [c.strip().lower().replace(" ", "_") for c in df_valuation.columns]
df_valuation.head()


,symbol,trailing_p/e,price/sales,price/book,peg,forward_p/e
0,MSFT,25.87,8.90,10.32,2.13,25.25
1,AMZN,NaN,NaN,NaN,NaN,NaN
2,TSLA,55.93,8.33,14.33,1.48,33.22
3,GOOGL,20.38,5.27,5.59,1.34,18.02
4,GOOG,20.57,5.32,5.64,1.36,18.18


In [17]:
df_valuation.columns = [c.replace("/", "_per_") for c in df_valuation.columns]
df_valuation.head()


,symbol,trailing_p_per_e,price_per_sales,price_per_book,peg,forward_p_per_e
0,MSFT,25.87,8.90,10.32,2.13,25.25
1,AMZN,NaN,NaN,NaN,NaN,NaN
2,TSLA,55.93,8.33,14.33,1.48,33.22
3,GOOGL,20.38,5.27,5.59,1.34,18.02
4,GOOG,20.57,5.32,5.64,1.36,18.18


# saya ambil tahun 2022

In [18]:
latest_year = fundamental["year"].max()
fundamental_latest = fundamental[fundamental["year"] == latest_year].copy()
fundamental_latest.head()


,symbol,year,assets,cash,equity,profit,revenue,roa,roe,profit_margin,cash_ratio,leverage
0,MSFT,2022,3.648400e+11,1.393100e+10,1.665420e+11,135620000000,198270000000,0.371725,0.814329,0.684017,0.038184,2.190679
1,AMZN,2022,4.205490e+11,3.622000e+10,1.382450e+11,197478000000,469822000000,0.469572,1.428464,0.420325,0.086126,3.042056
2,TSLA,2022,6.213100e+10,1.757600e+10,3.018900e+10,13606000000,53823000000,0.218989,0.450694,0.252792,0.282886,2.058068
3,GOOGL,2022,3.592680e+11,2.094500e+10,2.516350e+11,146698000000,257637000000,0.408325,0.582979,0.569398,0.058299,1.427735
4,GOOG,2022,3.592680e+11,2.094500e+10,2.516350e+11,146698000000,257637000000,0.408325,0.582979,0.569398,0.058299,1.427735


# join fundamental_latest itu untuk gabung ke stats dan valuation

In [19]:
fundamental_latest = fundamental_latest.merge(df_stats, on="symbol", how="left")
fundamental_latest = fundamental_latest.merge(df_valuation, on="symbol", how="left")

fundamental_latest.head()


,symbol,year,assets,cash,equity,profit,revenue,roa,roe,profit_margin_x,cash_ratio,leverage,return_on_equity,return_on_assets,profit_margin_y,trailing_p_per_e,price_per_sales,price_per_book,peg,forward_p_per_e
0,MSFT,2022,3.648400e+11,1.393100e+10,1.665420e+11,135620000000,198270000000,0.371725,0.814329,0.684017,0.038184,2.190679,42.88,15.22,34.37,25.87,8.90,10.32,2.13,25.25
1,AMZN,2022,4.205490e+11,3.622000e+10,1.382450e+11,197478000000,469822000000,0.469572,1.428464,0.420325,0.086126,3.042056,8.78,2.00,2.26,NaN,NaN,NaN,NaN,NaN
2,TSLA,2022,6.213100e+10,1.757600e+10,3.018900e+10,13606000000,53823000000,0.218989,0.450694,0.252792,0.282886,2.058068,32.24,11.72,14.95,55.93,8.33,14.33,1.48,33.22
3,GOOGL,2022,3.592680e+11,2.094500e+10,2.516350e+11,146698000000,257637000000,0.408325,0.582979,0.569398,0.058299,1.427735,26.89,13.92,23.75,20.38,5.27,5.59,1.34,18.02
4,GOOG,2022,3.592680e+11,2.094500e+10,2.516350e+11,146698000000,257637000000,0.408325,0.582979,0.569398,0.058299,1.427735,26.89,13.92,23.75,20.57,5.32,5.64,1.36,18.18


# simpan

In [21]:
fundamental.to_csv("data_cleaning/master_fundamental_all_years.csv", index=False)
fundamental_latest.to_csv("data_cleaning/master_fundamental_latest.csv", index=False)